## Imports

In [2]:
import mltlk
print(mltlk.__version__)
from mltlk import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam

0.1.33


## Load data
Load data, clean text and use Keras word vector embeddings preprocessing.

In [3]:
session = load_data("data/wikipedia_300.csv.gz",
                    preprocess="embeddings",
                    stopwords=["english","stopwords/custom.csv"],
                    clean_text="letters digits",
                    encode_categories=True,
                    embeddings_size=75,
                    embeddings_max_length=7500,
                    shuffle_data=True,
)

Info: Clean texts keeping letters and digits
Info: Categories encoded
Info: Load 180 stopwords from english, stopwords/custom.csv
Info: Vocabulary size is 53284
Info: 91.67% of sequences covered by max length 7500
Info: Loaded 300 examples in 2 categories


#### Show data stats

In [4]:
data_stats(session)

## Define Keras model
Builds the structure for the Keras model to use.

In [5]:
def get_model(session, mode):
    if mode == "nn":
        model = Sequential()
        model.add(Embedding(input_dim=session["vocab_size"], output_dim=session["embeddings_size"], input_length=session["max_length"]))
        model.add(Flatten())
        model.add(Dense(128, activation="relu", kernel_initializer="he_uniform"))
        model.add(Dropout(0.2))
        model.add(Dense(2, activation="softmax"))
        return model
    elif mode == "convnet":
        model = Sequential()
        model.add(Embedding(input_dim=session["vocab_size"], output_dim=session["embeddings_size"], input_length=session["max_length"]))
        model.add(Conv1D(filters=64, kernel_size=8, padding="same", activation="relu", kernel_initializer="he_uniform"))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(128, activation="relu", kernel_initializer="he_uniform"))
        model.add(Dropout(0.2))
        model.add(Dense(2, activation="softmax"))
        return model
    else:
        print("Unknown mode " + mode)
        return None

In [6]:
split_data(session,
           test_size=0.1,
           seed=4,
           stratify=True,
)

Info: Split data using 90% training data (270 samples) and 10% test data (30 samples) with seed 4 and stratify


## Neural Network model

In [7]:
evaluate_model(get_model(session, "nn"), 
               session, 
               reload=False,
               mode="split",
               categories=True,
               epochs=8,
               batch_size=32,
               loss="categorical_crossentropy",
               optimizer=Adam(learning_rate=0.01)
)

Info: Building and evaluating model using train-test split took 45.74 sec



Results,
Accuracy:,90.00%
F1-score:,89.90%
Precision:,91.67%
Recall:,90.00%


Programming (1),100.00%,15,15
Games (0),80.00%,12,15
Programming (1),25.00%,3,


## ConvNet model

In [9]:
evaluate_model(get_model(session, "convnet"), 
               session, 
               reload=True,
               mode="split",
               categories=True,
               epochs=8,
               batch_size=32,
               loss="categorical_crossentropy",
               optimizer=Adam(learning_rate=0.01)
)

Info: Building and evaluating model using train-test split took 48.37 sec



Results,
Accuracy:,90.00%
F1-score:,89.99%
Precision:,90.18%
Recall:,90.00%


Programming (1),93.33%,14,15
Games (0),7.14%,1,
Games (0),86.67%,13,15
Programming (1),15.38%,2,
